In [ ]:
# !pip install reformer_pytorch
!bash ./scripts/Weather_script/Autoformer.sh


/opt/notebooks/Autoformer-main
Args in experiment:
Namespace(activation='gelu', batch_size=32, bucket_size=4, c_out=21, checkpoints='./checkpoints/', d_ff=2048, d_layers=1, d_model=512, data='custom', data_path='weather.csv', dec_in=21, des='Exp', devices='0,1,2', distil=True, do_predict=False, dropout=0.05, e_layers=2, embed='timeF', enc_in=21, factor=3, features='M', freq='h', gpu=0, is_training=1, itr=1, label_len=48, learning_rate=0.0001, loss='mse', lradj='type1', model='Autoformer', model_id='weather_96_96', moving_avg=25, n_hashes=4, n_heads=8, num_workers=10, output_attention=False, patience=3, pred_len=96, root_path='./dataset/weather/', seq_len=96, target='OT', train_epochs=2, use_amp=False, use_gpu=True, use_multi_gpu=False)
-------
Use GPU: cuda:0
>>>>>>>start training : weather_96_96_Autoformer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 36696
val 5175
test 10444
	iters: 100, epoch: 1 | loss: 1.4534090
	speed:

In [1]:
!nvidia-smi

Mon May  1 14:39:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:3B:00.0 Off |                  Off |
| 26%   35C    P8    11W / 180W |    195MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro P5000        Off  | 00000000:AF:00.0 Off |                  Off |
| 26%   

In [15]:
import requests
import pandas as pd
import json

# API key
API_KEY = "5c2dcc46834a46c1837a12eaadfe275a"

# Request parameters
lat = -33.86
lon = 151.209
start_date = "2023-04-01"
end_date = "2023-05-01"
tz = "local"

# API request URL
url = f"https://api.weatherbit.io/v2.0/history/subhourly?lat={lat}&lon={lon}&start_date={start_date}&end_date={end_date}&tz={tz}&key={API_KEY}"

# Send API request and get response
response = requests.get(url)

# Check response status code
if response.status_code == 200:
    # Extract data from response
    data = response.json()
    # Print data
    # print(data)
    
    with open("data11.json", "w") as outfile:
        json.dump(data, outfile)

    
# # Convert JSON data to a DataFrame
#     df = pd.DataFrame.from_dict([data])

# # Save data to CSV file
#     df.to_csv('data.csv', index=False)
    
else:
    # Print error message
    print(f"Error {response.status_code}: {response.text}")

In [9]:
# loop api for the limit of one month call per time
# To obtain past 5 years weather data from weatherbit api(15 mins interval)
import requests
import pandas as pd
import json
import datetime
import time
# API key
API_KEY = "5c2dcc46834a46c1837a12eaadfe275a"

# Request parameters
lat = -33.86
lon = 151.209
tz = "local"


# Start and end dates
start_date = datetime.date(2018, 5, 1)
end_date = datetime.date(2022, 5, 1)

# Loop through each month and make API call
current_date = start_date
i = 0
while current_date < end_date:
    # Calculate start and end dates for current month
    current_month = current_date.strftime("%Y-%m")
    start_of_month = current_date.strftime("%Y-%m-%d")
    end_of_month = (current_date.replace(day=28) + datetime.timedelta(days=4)).strftime("%Y-%m-%d")
    
    
    url = f"https://api.weatherbit.io/v2.0/history/subhourly?lat={lat}&lon={lon}&start_date={start_of_month}&end_date={end_of_month}&tz={tz}&key={API_KEY}"

    # Send API request and get response
    response = requests.get(url)

    # Check response status code
    if response.status_code == 200:
    # Extract data from response
        data = response.json()
    # Print data
    # print(data)
    
        with open(f"jsonData1/data{str(i)}.json", "w") as outfile:
            json.dump(data, outfile)
    else:
        print(f"Error {response.status_code}: {response.text}")

    
    # Move to next month
    current_date = current_date.replace(day=28) + datetime.timedelta(days=4)
    i = i+1

    # Wait for some time before the next API call
    time.sleep(1) # add some delay to avoid overloading the API server

    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [ ]:
import json
import pandas


def read_json(filename: str) -> dict:

	try:
		with open(filename, "r") as f:
			data = json.loads(f.read())
	except:
		raise Exception(f"Reading {filename} file encountered an error")

	return data


def create_dataframe(data: list) -> pandas.DataFrame:

	# Declare an empty dataframe to append records
	dataframe = pandas.DataFrame()

	# Looping through each record
	for d in data:
		
		# Normalize the column levels
		record = pandas.json_normalize(d)
		
		# Append it to the dataframe
		dataframe = pd.concat([dataframe, record], ignore_index=True)



	return dataframe


def main():
	# Read the JSON file as python dictionary
	data = read_json(filename="data.json")
	# print(data['data'])

	# Generate the dataframe for the array items in
	# details key
	dataframe = create_dataframe(data=data['data'])

	# Renaming columns of the dataframe
	print("Normalized Columns:", dataframe.columns.to_list())

# 	dataframe.rename(columns={
# 		"results.school": "school",
# 		"results.high_school": "high_school",
# 		"results.graduation": "graduation",
# 		"education.graduation.major": "grad_major",
# 		"education.graduation.minor": "grad_minor"
# 	}, inplace=True)

	print("Renamed Columns:", dataframe.columns.to_list())

	# Convert dataframe to CSV
	dataframe.to_csv("details.csv", index=False)


if __name__ == '__main__':
	main()


Normalized Columns: ['app_temp', 'azimuth', 'clouds', 'dewpt', 'dhi', 'dni', 'elev_angle', 'ghi', 'pod', 'precip_rate', 'pres', 'revision_status', 'rh', 'slp', 'snow_rate', 'solar_rad', 'temp', 'timestamp_local', 'timestamp_utc', 'ts', 'uv', 'vis', 'wind_dir', 'wind_gust_spd', 'wind_spd', 'weather.description', 'weather.code', 'weather.icon']
Renamed Columns: ['app_temp', 'azimuth', 'clouds', 'dewpt', 'dhi', 'dni', 'elev_angle', 'ghi', 'pod', 'precip_rate', 'pres', 'revision_status', 'rh', 'slp', 'snow_rate', 'solar_rad', 'temp', 'timestamp_local', 'timestamp_utc', 'ts', 'uv', 'vis', 'wind_dir', 'wind_gust_spd', 'wind_spd', 'weather.description', 'weather.code', 'weather.icon']


In [13]:
# convert all json data into single csv file
import json
import pandas
  
  
def read_json(filename: str) -> dict:
  
    try:
        with open(filename, "r") as f:
            data = json.loads(f.read())
    except:
        raise Exception(f"Reading {filename} file encountered an error")
  
    return data
  
  
def create_dataframe(data: list) -> pandas.DataFrame:
  
    # Declare an empty dataframe to append records
    dataframe = pandas.DataFrame()
  
    # Looping through each record
    for d in data:
          
        # Normalize the column levels
        record = pandas.json_normalize(d)
        record = record.reindex(sorted(record.columns), axis=1)

          
        # Append it to the dataframe 
        dataframe = pandas.concat([dataframe, record], ignore_index=True)
  
    return dataframe
  
  
def main():
    # Read the JSON file as python dictionary 
    data = read_json(filename="jsonData1/data0.json")
    dataframe = create_dataframe(data=data['data'])
    dataframe.to_csv("details1.csv", index=False)
    for i in range(47):
        data = read_json(filename=f"jsonData1/data{i+1}.json")

        # Generate the dataframe for the array items in 
        # details key 
        dataframe = create_dataframe(data=data['data'])
        print(dataframe.shape)

        # Renaming columns of the dataframe 
        # print("Normalized Columns:", dataframe.columns.to_list())
        print(i)


        # Convert dataframe to CSV
        dataframe.to_csv("details1.csv", mode='a', index=False, header=False)
  
  
if __name__ == '__main__':
    main()

(2976, 28)
0
(2880, 28)
1
(2976, 28)
2
(2976, 28)
3
(2876, 28)
4
(2976, 28)
5
(2880, 28)
6
(2976, 28)
7
(2976, 28)
8
(2688, 28)
9
(2980, 28)
10
(2880, 28)
11
(2976, 28)
12
(2880, 28)
13
(2976, 28)
14
(2976, 28)
15
(2876, 28)
16
(2976, 28)
17
(2880, 28)
18
(2976, 28)
19
(2976, 28)
20
(2784, 28)
21
(2980, 28)
22
(2880, 28)
23
(2976, 28)
24
(2880, 28)
25
(2976, 28)
26
(2976, 28)
27
(2876, 28)
28
(2976, 28)
29
(2880, 28)
30
(2976, 28)
31
(2976, 28)
32
(2688, 28)
33
(2980, 28)
34
(2880, 28)
35
(2976, 28)
36
(2880, 28)
37
(2976, 28)
38
(2976, 28)
39
(2876, 28)
40
(2976, 28)
41
(2880, 28)
42
(2976, 28)
43
(2976, 28)
44
(2688, 28)
45
(2980, 28)
46


In [14]:
# 删除重复时间数据
import csv

# 设置要删除的行数范围
start_row = 140257
end_row = 140352

# 打开CSV文件并读取数据
with open('details1.csv', 'r') as infile:
    reader = csv.reader(infile)
    data = [row for i, row in enumerate(reader) if i < start_row or i > end_row]

# 将结果写回到原始CSV文件或者写入新的CSV文件
with open('details2.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerows(data)

In [15]:
# preprocess csv data to remove useless columns and also one hot encode 
# weather desc string with float number
import pandas as pd

df = pd.read_csv('details1.csv', low_memory=False)
one_hot_encoded = pd.get_dummies(df['weather.description'], dtype=float)
# print(one_hot_encoded)
df = pd.concat([df, one_hot_encoded], axis=1)
# df.head()
print(df.columns)
df = df.drop(['timestamp_utc', 'weather.code','weather.description',
       'weather.icon','revision_status','ts', 'pod'], axis=1)
df = df.rename(columns={'timestamp_local': 'date'})

print(len(df.columns))
df.to_csv("processedData1.csv", index=False)


Index(['app_temp', 'azimuth', 'clouds', 'dewpt', 'dhi', 'dni', 'elev_angle',
       'ghi', 'pod', 'precip_rate', 'pres', 'revision_status', 'rh', 'slp',
       'snow_rate', 'solar_rad', 'temp', 'timestamp_local', 'timestamp_utc',
       'ts', 'uv', 'vis', 'weather.code', 'weather.description',
       'weather.icon', 'wind_dir', 'wind_gust_spd', 'wind_spd',
       'Broken clouds', 'Clear Sky', 'Few clouds', 'Fog', 'Haze', 'Heavy rain',
       'Light rain', 'Moderate rain', 'Overcast clouds', 'Scattered clouds',
       'Thunderstorm with heavy rain'],
      dtype='object')
32


In [1]:
# append 2022-2023data to details2(2018-2022)data
# processedData1.csv 2018-2022
# processedData.csv 2022-2023
import pandas as pd
df1 = pd.read_csv('dataset/weather/processedData1.csv')
df2 = pd.read_csv('dataset/weather/processedData.csv')

# Concatenate the two dataframes vertically
result = pd.concat([df1, df2], ignore_index=True)

# Write the result to a new CSV file
result.to_csv('dataset/weather/data.csv', index=False)

In [2]:

import pandas as pd
df1 = pd.read_csv('dataset/weather/processedData1.csv')
print(df1.columns)
df1.to_csv('col.csv')

Index(['app_temp', 'azimuth', 'clouds', 'dewpt', 'dhi', 'dni', 'elev_angle',
       'ghi', 'precip_rate', 'pres', 'rh', 'slp', 'snow_rate', 'solar_rad',
       'temp', 'date', 'uv', 'vis', 'wind_dir', 'wind_gust_spd', 'wind_spd',
       'Broken clouds', 'Clear Sky', 'Few clouds', 'Fog', 'Haze', 'Heavy rain',
       'Light rain', 'Moderate rain', 'Overcast clouds', 'Scattered clouds',
       'Thunderstorm with heavy rain'],
      dtype='object')


In [2]:
# train the model
!bash ./scripts/Weather_script/myAutoformer.sh


/opt/notebooks/Autoformer-main
Args in experiment:
Namespace(activation='gelu', batch_size=32, bucket_size=4, c_out=31, checkpoints='./checkpoints/', d_ff=2048, d_layers=1, d_model=512, data='custom', data_path='data.csv', dec_in=31, des='Exp', devices='0,1,2', distil=True, do_predict=False, dropout=0.05, e_layers=2, embed='timeF', enc_in=31, factor=3, features='M', freq='h', gpu=0, is_training=1, itr=1, label_len=48, learning_rate=0.0001, loss='mse', lradj='type1', model='Autoformer', model_id='weather_96_96', moving_avg=25, n_hashes=4, n_heads=8, num_workers=10, output_attention=False, patience=3, pred_len=96, root_path='./dataset/weather/', seq_len=96, target='temp', train_epochs=5, use_amp=False, use_gpu=True, use_multi_gpu=False)
-------
Use GPU: cuda:0
>>>>>>>start training : weather_96_96_Autoformer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 122583
val 17445
test 34983
	iters: 100, epoch: 1 | loss: 0.5753578
	speed

In [1]:
# predict
!bash ./scripts/Weather_script/myAutoformer.sh

/opt/notebooks/Autoformer-main
Args in experiment:
Namespace(activation='gelu', batch_size=32, bucket_size=4, c_out=31, checkpoints='./checkpoints/', d_ff=2048, d_layers=1, d_model=512, data='custom', data_path='data.csv', dec_in=31, des='Exp', devices='0,1,2', distil=True, do_predict=False, dropout=0.05, e_layers=2, embed='timeF', enc_in=31, factor=3, features='M', freq='h', gpu=0, is_training=1, itr=1, label_len=48, learning_rate=0.0001, loss='mse', lradj='type1', model='Autoformer', model_id='weather_96_96', moving_avg=25, n_hashes=4, n_heads=8, num_workers=10, output_attention=False, patience=3, pred_len=96, root_path='./dataset/weather/', seq_len=96, target='temp', train_epochs=10, use_amp=False, use_gpu=True, use_multi_gpu=False)
-------
Use GPU: cuda:0
>>>>>>>start training : weather_96_96_Autoformer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 122583
val 17445
test 34983
	iters: 100, epoch: 1 | loss: 0.5753577
	spee

In [11]:
import torch
batch_x = torch.randn([1, 96, 31]).cpu()
batch_x_mark = torch.randn([1, 96, 4]).cpu()
dec_inp = torch.randn([1, 144, 31]).cpu()
batch_y_mark = torch.randn([1, 144, 4]).cpu()

model = torch.load("checkpoints/weather_96_96_Autoformer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0/bestModel.pt").cpu()
model.eval()

# print(model)
traced_model = torch.jit.trace(model,  example_inputs=(batch_x, batch_x_mark, dec_inp, batch_y_mark))
print(traced_model)
# traced_model.save("tracedModel.pt")


# torch.onnx.export(traced_model,               # model being run
#                   (batch_x, batch_x_mark, dec_inp, batch_y_mark),                         # model input (or a tuple for multiple inputs)
#                   "model.onnx",   # where to save the model (can be a file or file-like object)
#                   export_params=True,        # store the trained parameter weights inside the model file
#                   cj          # the ONNX version to export the model to
#                   do_constant_folding=True,  # whether to execute constant folding for optimization
#                   input_names = ['input'],   # the model's input names
#                   output_names = ['output'])


TypeError: 'ScriptMethodStub' object is not callable

In [7]:
output = traced_model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
print(output.shape)

torch.Size([1, 96, 31])


In [ ]:
!bash ./scripts/Weather_script/myAutoformer.sh


/opt/notebooks/Autoformer-main
Args in experiment:
Namespace(activation='gelu', batch_size=32, bucket_size=4, c_out=31, checkpoints='./checkpoints/', d_ff=2048, d_layers=1, d_model=512, data='custom', data_path='data.csv', dec_in=31, des='Exp', devices='0,1,2', distil=True, do_predict=True, dropout=0.05, e_layers=2, embed='timeF', enc_in=31, factor=3, features='M', freq='m', gpu=0, is_training=1, itr=1, label_len=48, learning_rate=0.0001, loss='mse', lradj='type1', model='Autoformer', model_id='weather_96_96', moving_avg=25, n_hashes=4, n_heads=8, num_workers=10, output_attention=False, patience=3, pred_len=96, root_path='./dataset/weather/', seq_len=96, target='temp', train_epochs=10, use_amp=False, use_gpu=True, use_multi_gpu=False)
-------
Use GPU: cuda:0
>>>>>>>start training : weather_96_96_Autoformer_custom_ftM_sl96_ll48_pl96_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 122583
val 17445
test 34983
	iters: 100, epoch: 1 | loss: 0.5649285
	speed